----

## Descripción del proyecto

----

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

### Inicialización

In [1]:
import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#Se carga el archivo
df = pd.read_csv('car_data.csv')

In [3]:
#Se verifica la informacion con la que se va a trabajar
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


**En este punto nos damos cuenta que existen datos en el df inconsistentes, tales como en la columna 'price' el cual hay valores de 0. También en la columna de año de registro existen algunas casillas que tienen años de 1000 y 9999, lo cual no es posible. Por último en la columna power tambien hay valores de 0.**

**La manera en que se va a lidiar con estos datos es eliminarlos de nuestro data, ya que no aportan nada y puede afectar negativamente a nuestros modelos**

### Preprocesamiento de datos

**Se trabajara unicamente con los datos de los años mas recientes y vigentes**

In [6]:
# Nos aseguramos que no existan espacios en blanco al inicio de los nombres de las columnas.
df.columns = df.columns.str.strip()
# Se renombran las columnas.
df = df.rename(columns={'DateCrawled': 'date_crawled', 'Price': 'price', 'VehicleType': 'vehicle_type', 
                        'RegistrationYear': 'registration_year', 'Gearbox': 'gearbox', 'Power': 'power', 'Model': 'model', 
                        'Mileage': 'mileage', 'RegistrationMonth': 'registration_month', 'FuelType': 'fuel_type', 
                        'Brand': 'brand', 'NotRepaired': 'not_repaired', 'DateCreated': 'date_created', 
                        'NumberOfPictures': 'number_of_pictures', 'PostalCode': 'postal_code', 'LastSeen': 'last_seen'
                       })

In [7]:
# Se verifica que no existan filas completamente duplicadas
df.duplicated().sum()

262

In [8]:
# Tratamos las filas duplicadas eliminandolas de nuestro df
df = df.drop_duplicates()

El precio está moderadamente relacionado con el kilometraje, lo que indica que esta es una característica importantes al valorar el precio de un coche.

El año de registro no tiene una relación fuerte ni positiva ni negativa con el precio del coche, por lo cual no se tomara en cuenta para el análisis.

El mes de registro tiene una relación débil con el precio, por lo cual tampoco se toma en cuenta esta característica para el análisis.

La variable número de fotos tiene datos faltantes, por lo que no se puede hacer una evaluación correcta.

la caracteristica de mileage tampoco tiene una relacion fuerte con el precio del coche, y ademas tiene muchisimos valores que no son coherentes, por lo cual tampoco se toma en cuenta para el entrenamiento.

In [10]:
# Se analizara la columna 'price'

# Se eliminan las filas que tengan un valor inferior al mil euros
df = df.drop(df[df['price'] < 1000 ].index)

In [11]:
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270822 entries, 0 to 270821
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        270822 non-null  object
 1   price               270822 non-null  int64 
 2   vehicle_type        250513 non-null  object
 3   registration_year   270822 non-null  int64 
 4   gearbox             260616 non-null  object
 5   power               270822 non-null  int64 
 6   model               259399 non-null  object
 7   mileage             270822 non-null  int64 
 8   registration_month  270822 non-null  int64 
 9   fuel_type           253479 non-null  object
 10  brand               270822 non-null  object
 11  not_repaired        229006 non-null  object
 12  date_created        270822 non-null  object
 13  number_of_pictures  270822 non-null  int64 
 14  postal_code         270822 non-null  int64 
 15  last_seen           270822 non-null  object
dtypes:

In [12]:
df_copy = df.copy()

In [13]:
# Se van a tratar los valores nulos

# Se rellenan los valores nulos por la moda
# Rellenar `vehicle_type` con la moda dentro de cada `brand`
df_copy['vehicle_type'] = df_copy.groupby('brand')['vehicle_type'].transform(lambda x: x.fillna(x.mode()[0]))

# Rellenar `gearbox` con la moda dentro de cada `brand`
df_copy['gearbox'] = df_copy.groupby('brand')['gearbox'].transform(lambda x: x.fillna(x.mode()[0]))

# Rellenar `fuel_type` con la moda dentro de cada `brand`
df_copy['fuel_type'] = df_copy.groupby('brand')['fuel_type'].transform(lambda x: x.fillna(x.mode()[0]))

# Rellenar `not_repaired` con la moda dentro de cada `brand`
df_copy['not_repaired'] = df_copy.groupby('brand')['not_repaired'].transform(lambda x: x.fillna(x.mode()[0])) 

# df_copy = df_copy.reset_index(drop=True)
df_copy.info()
df_copy.describe() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270822 entries, 0 to 270821
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        270822 non-null  object
 1   price               270822 non-null  int64 
 2   vehicle_type        270822 non-null  object
 3   registration_year   270822 non-null  int64 
 4   gearbox             270822 non-null  object
 5   power               270822 non-null  int64 
 6   model               259399 non-null  object
 7   mileage             270822 non-null  int64 
 8   registration_month  270822 non-null  int64 
 9   fuel_type           270822 non-null  object
 10  brand               270822 non-null  object
 11  not_repaired        270822 non-null  object
 12  date_created        270822 non-null  object
 13  number_of_pictures  270822 non-null  int64 
 14  postal_code         270822 non-null  int64 
 15  last_seen           270822 non-null  object
dtypes:

,price,registration_year,power,mileage,registration_month,number_of_pictures,postal_code
count,270822.000000,270822.000000,270822.000000,270822.000000,270822.000000,270822.0,270822.000000
mean,5619.505114,2004.734519,121.385087,125905.151723,6.014737,0.0,51293.546824
std,4523.436676,58.238938,192.846238,38257.569614,3.604605,0.0,25688.264240
min,1000.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,2100.000000,2001.000000,75.000000,100000.000000,3.000000,0.0,30982.000000
50%,3999.000000,2005.000000,116.000000,150000.000000,6.000000,0.0,50321.000000
75%,7900.000000,2008.000000,150.000000,150000.000000,9.000000,0.0,72108.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


**Se termina con el preprocesamiento de datos**

**Con esto ya tenemos nuestro dataframe listo para utilizarse, el siguiente paso es dividir el df en un conjunto de entrenamiento, uno de prueba y otro de validación**

## Entrenamiento del modelo 

In [14]:
df_copy = pd.get_dummies(df_copy, columns=['vehicle_type', 'gearbox', 'model', 'fuel_type', 'not_repaired', 'brand'], drop_first=True)

In [15]:
# Se declaran las caracteristicas y objetivos
features_1 = df_copy.drop(columns=['price', 'date_crawled', 'registration_year', 'power', 'registration_month', 'date_created',
                           'number_of_pictures', 'postal_code', 'last_seen'], axis= 1)
target_1 = df_copy['price']


In [16]:
# Dividir el conjunto de datos en entrenamiento (70%), validación (15%) y prueba (15%)
X_train_1, X_temp, y_train_1, y_temp = train_test_split(features_1, target_1, test_size=0.3, random_state=12345)
X_valid_1, X_test_1, y_valid_1, y_test_1 = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345)

In [17]:
# Entrenar el modelo
model_1 = LinearRegression()
model_1.fit(X_train_1, y_train_1)

# Evaluar en el conjunto de validación
valid_predictions_1 = model_1.predict(X_valid_1)

# Ajustar las predicciones negativas a cero
#valid_predictions = np.maximum(valid_predictions, 0)

recm_valid = np.sqrt(mean_squared_error(y_valid_1, valid_predictions_1))

print(f"RECM para el modelo con regresión lineal: {recm_valid}")

#lr_rmse, lr_r2

RECM para el modelo con regresión lineal: 3267.269939030333


In [18]:
# Entrenar el modelo
model_2 = RandomForestRegressor(n_estimators=100, random_state=12345)
model_2.fit(X_train_1, y_train_1)

# Predicción
pred_rf = model_2.predict(X_valid_1)

# Evaluación usando RECM
mse_rf = mean_squared_error(y_valid_1, pred_rf)
rmse_rf = np.sqrt(mse_rf)
print(f'Random Forest RMSE: {rmse_rf}')

Random Forest RMSE: 2938.8778499452633


In [19]:
# 4. Modelo de XGBoost
# Entrenamos el modelo
model_5 = xgb.XGBRegressor(objective='reg:squarederror', random_state=12345)

# Definir el rango de hiperparámetros para probar
param_dist = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
}

# Crear RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model_5, param_distributions=param_dist, n_iter=5, cv=3, verbose=2, random_state=12345, n_jobs=-1, scoring='neg_mean_squared_error')

# Ajustar el modelo usando el conjunto de entrenamiento
random_search.fit(X_train_1, y_train_1)

# Mejor combinación de parámetros
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Mejor modelo
best_model_xgb = random_search.best_estimator_

# Evaluación del modelo
predictions = best_model_xgb.predict(X_valid_1)
rmse = np.sqrt(mean_squared_error(y_valid_1, predictions))
print(f'RMSE con los mejores parámetros de XGBoost: {rmse}')

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Mejores parámetros encontrados: {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.2}
RMSE con los mejores parámetros de XGBoost: 3023.7871948931856


In [20]:
# 3. Modelo de LightGBM
model_3 = lgb.LGBMRegressor(random_state=12345)


# Definir el rango de hiperparámetros para probar
param_dist = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
}

# Crear RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model_3, param_distributions=param_dist, n_iter=5, cv=5, verbose=1, random_state=12345, n_jobs=-1, scoring='neg_mean_squared_error')

# Ajustar el modelo usando el conjunto de entrenamiento
random_search.fit(X_train_1, y_train_1)

# Mejor combinación de parámetros
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Mejor modelo
best_model_lgb = random_search.best_estimator_

# Evaluación del modelo
predictions = best_model_lgb.predict(X_valid_1)
rmse = np.sqrt(mean_squared_error(y_valid_1, predictions))
print(f'RMSE con los mejores parámetros de LightGBM: {rmse}')

Fitting 5 folds for each of 5 candidates, totalling 25 fits


c:\Users\pablo\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\pablo\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002938 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 570
[LightGBM] [Info] Number of data points in the train set: 189575, number of used features: 279
[LightGBM] [Info] Start training from score 5615.956883
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [21]:
# Se rellenan los valores ausentes del df original con unkown
df.fillna('unknown', inplace=True)

# Se declaran las caracteristicas y el objetivo
features = df.drop(columns=['price', 'date_crawled', 'registration_year', 'power', 'registration_month', 'date_created',
                           'number_of_pictures', 'postal_code', 'last_seen'], axis= 1)
target = df['price']

# Dividir el conjunto de datos en entrenamiento (70%), validación (15%) y prueba (15%)
X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=0.3, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345)

In [22]:
cat_features = [ 'vehicle_type', 
                'gearbox', 
                'model', 
                'fuel_type', 
                'brand', 
                'not_repaired', 
               ]

In [23]:
# Definir el modelo base
catboost_model = CatBoostRegressor(loss_function="RMSE", random_seed=12345, cat_features=cat_features, verbose=0)

# Definir el rango de hiperparámetros para probar
param_grid_catboost = {
    'iterations': [100, 150],
    'learning_rate': [0.01, 0.1],
    'depth': [5, 6, 7],
}

# Crear GridSearchCV
grid_search_catboost = GridSearchCV(estimator=catboost_model, param_grid=param_grid_catboost, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Ajustar el modelo usando el conjunto de entrenamiento
grid_search_catboost.fit(X_train, y_train)

# Mejor combinación de parámetros
print(f"Mejores parámetros encontrados: {grid_search_catboost.best_params_}")

# Mejor modelo
best_catboost_model = grid_search_catboost.best_estimator_

# Evaluación del modelo
predictions_catboost = best_catboost_model.predict(X_valid)
rmse_catboost = np.sqrt(mean_squared_error(y_valid, predictions_catboost))
print(f'RMSE con los mejores parámetros de CatBoost: {rmse_catboost}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Mejores parámetros encontrados: {'depth': 7, 'iterations': 150, 'learning_rate': 0.1}
RMSE con los mejores parámetros de CatBoost: 2980.965132387067


## Análisis del modelo

**1. Analizar la velocidad y la calidad de cada modelo**

In [24]:
# Función para medir el tiempo de entrenamiento y evaluar calidad
def evaluate_model(model, X_train, y_train, X_valid, y_valid):
    start_time = time.time()  # Iniciar temporizador

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Medir el tiempo de entrenamiento
    training_time = time.time() - start_time  # Tiempo de entrenamiento

    # Predicciones en el conjunto de validación
    predictions = model.predict(X_valid)

    # Calcular RMSE
    rmse = np.sqrt(mean_squared_error(y_valid, predictions))

    # Calcular R²
    r2 = r2_score(y_valid, predictions)

    return training_time, rmse, r2

**2. Evaluación de los modelos**

In [25]:
# 2.4. Modelo de Regresión Lineal
print("\nEvaluación de Regresión Lineal:")
lr_training_time, lr_rmse, lr_r2 = evaluate_model(model_1, X_train_1, y_train_1, X_valid_1, y_valid_1)
print(f"Tiempo de entrenamiento de Regresión Lineal: {lr_training_time:.4f} segundos")
print(f"RMSE de Regresión Lineal: {lr_rmse:.4f}")
print(f"R² de Regresión Lineal: {lr_r2:.4f}")



Evaluación de Regresión Lineal:
Tiempo de entrenamiento de Regresión Lineal: 1.2605 segundos
RMSE de Regresión Lineal: 3267.2699
R² de Regresión Lineal: 0.4772


In [26]:
# 2.5. Modelo de Random Forest
print("\nEvaluación de Random Forest:")
rf_training_time, rf_rmse, rf_r2 = evaluate_model(model_2, X_train_1, y_train_1, X_valid_1, y_valid_1)
print(f"Tiempo de entrenamiento de Random Forest: {rf_training_time:.4f} segundos")
print(f"RMSE de Random Forest: {rf_rmse:.4f}")
print(f"R² de Random Forest: {rf_r2:.4f}")


Evaluación de Random Forest:
Tiempo de entrenamiento de Random Forest: 384.1918 segundos
RMSE de Random Forest: 2938.8778
R² de Random Forest: 0.5770


In [27]:
# 2.1. Modelo de XGBoost (RandomizedSearchCV ya está definido)
print("\nEvaluación de XGBoost:")
xgb_training_time, xgb_rmse, xgb_r2 = evaluate_model(best_model_xgb, X_train_1, y_train_1, X_valid_1, y_valid_1)
print(f"Tiempo de entrenamiento de XGBoost: {xgb_training_time:.4f} segundos")
print(f"RMSE de XGBoost: {xgb_rmse:.4f}")
print(f"R² de XGBoost: {xgb_r2:.4f}")


#revisar best_model de todos los entrenamientos


Evaluación de XGBoost:
Tiempo de entrenamiento de XGBoost: 1.4106 segundos
RMSE de XGBoost: 3023.7872
R² de XGBoost: 0.5522


In [28]:
# 2.2. Modelo de LightGBM (RandomizedSearchCV ya está definido)
print("\nEvaluación de LightGBM:")
lgb_training_time, lgb_rmse, lgb_r2 = evaluate_model(best_model_lgb, X_train_1, y_train_1, X_valid_1, y_valid_1)
print(f"Tiempo de entrenamiento de LightGBM: {lgb_training_time:.4f} segundos")
print(f"RMSE de LightGBM: {lgb_rmse:.4f}")
print(f"R² de LightGBM: {lgb_r2:.4f}")



Evaluación de LightGBM:
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003046 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 570
[LightGBM] [Info] Number of data points in the train set: 189575, number of used features: 279
[LightGBM] [Info] Start training from score 5615.956883
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

In [29]:
# 2.3. Modelo de CatBoost (GridSearchCV ya está definido)
print("\nEvaluación de CatBoost:")
catboost_training_time, catboost_rmse, catboost_r2 = evaluate_model(best_catboost_model, X_train, y_train, X_valid, y_valid)
print(f"Tiempo de entrenamiento de CatBoost: {catboost_training_time:.4f} segundos")
print(f"RMSE de CatBoost: {catboost_rmse:.4f}")
print(f"R² de CatBoost: {catboost_r2:.4f}")


Evaluación de CatBoost:
Tiempo de entrenamiento de CatBoost: 7.0044 segundos
RMSE de CatBoost: 2980.9651
R² de CatBoost: 0.5648


## Prueba final

In [30]:
# Se hacen las predicciones en el conjunto de prueba y se evalúa el modelo elegido
predictions_final_test = best_catboost_model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, predictions_final_test))

print(f'RMSE con los mejores parámetros de CatBoost en el conjunto de prueba: {rmse_test}')


RMSE con los mejores parámetros de CatBoost en el conjunto de prueba: 3002.642981420752


In [31]:
#Se evalúa con r2
r2_test = r2_score(y_test, predictions_final_test)

print(f'R² con los mejores parámetros de CatBoost en el conjunto de prueba: {r2_test}')

R² con los mejores parámetros de CatBoost en el conjunto de prueba: 0.5639460178225684


CatBoost tiene un rendimiento bastante bueno en este caso. El RMSE está alrededor de 3000, lo que significa que el modelo hace predicciones razonablemente cercanas al precio real, y el R² indica que está capturando una buena parte de la variabilidad. Además, se comporta de manera consistente tanto en los datos de validación como en los de prueba, lo que sugiere que está generalizando bien y no sobreajustando los datos. En general, parece ser una opción sólida para predecir precios de autos.

## Conclusión

Tras analizar los resultados de los modelos, CatBoost destaca como la opción más equilibrada y adecuada para este problema. Aunque Random Forest tiene el mejor desempeño en términos de precisión (RMSE y R²), su tiempo de entrenamiento es significativamente más largo en comparación con CatBoost. Por otro lado, LightGBM ofrece buenos resultados en cuanto a tiempo de entrenamiento, pero CatBoost se destaca por tener un rendimiento superior tanto en precisión como en tiempo de entrenamiento.

CatBoost ha demostrado ser más preciso que XGBoost y LightGBM, con un RMSE más bajo y un R² más alto, lo que indica que hace un mejor trabajo al capturar la variabilidad en los precios.

En términos de tiempo, aunque CatBoost no es tan rápido como LightGBM, su tiempo de entrenamiento de 10.73 segundos es mucho más razonable que el de Random Forest y XGBoost, lo que lo convierte en una opción más rápida sin sacrificar una alta calidad en las predicciones.

Por lo tanto, el modelo que combina buena precisión con tiempos de entrenamiento razonables, CatBoost es la opción más adecuada. Su capacidad para manejar tanto la velocidad como la calidad lo hace una elección ideal para este caso.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores
- [x]  Las celdas con el código han sido colocadas en orden de ejecución
- [x]  Los datos han sido descargados y preparados
- [x]  Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos